In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

Exception ignored in: <_io.FileIO name='data/bags/instances.pkl' mode='rb' closefd=True>


In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(31943, 1035)

# run

## initialize log

In [6]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [7]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 10000 20000 30000 0 0 

## run

In [8]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
   Time  Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0   129   0  0    485    461   457  0.05  0.36  0.46  0.86  0.78
1   130   1  0    482    459   455  0.05  0.37  0.50  0.82  0.73
2   132   2  0    478    456   453  0.05  0.41  0.52  0.78  0.67
3   132   3  0    475    455   453  0.05  0.47  0.53  0.75  0.54
4   133   4  0    470    450   448  0.05  0.49  0.53  0.71  0.49
5   146   5  0    456    438   435  0.05  0.51  0.55  0.65  0.47
6   151   6  0    432    417   413  0.05  0.52  0.56  0.61  0.46
7   152   7  0    407    395   394  0.05  0.53  0.60  0.57  0.44
8   150   8  0    400    397   394  0.05  0.52  0.61  0.53  0.42
9   150   9  0    402    403   394  0.05  0.52  0.66  0.47  0.34
10  162  10  0    395    392   383  0.05  0.49  0.65  0.48  0.40
11  167  11  0    378    372   368  0.05  0.43  0.65  0.52  0.38
12  162  12  0    367    358   358  0.05  0.38  0.67  0.54  0.26
13  191  13  0    357    346   350  0.06  0.44  0.64  0.67  0.41
14  211  14  0    346    336   339  0.07  0.46  0.66  0.63  0.35
15  211  15  0    348    339   339  0.08  0.44  0.64  0.58  0.31
16  211  16  0    352    346   339  0.09  0.47  0.61  0.64  0.37
17  220  17  0    345    335   332  0.09  0.51  0.60  0.58  0.37
18  221  18  0    347    343   332  0.10  0.51  0.59  0.51  0.33
19  234  19  0    342    339   332  0.10  0.56  0.59  0.59  0.38
20  233  20  0    338    336   332  0.10  0.58  0.58  0.58  0.34
21  231  21  0    345    344   332  0.10  0.58  0.55  0.59  0.33
22  231  22  0    352    358   332  0.09  0.59  0.53  0.58  0.34
23  244  23  0    351    354   332  0.09  0.59  0.54  0.64  0.40
24  239  24  0    345    348   332  0.09  0.61  0.55  0.61  0.35
25  241  25  0    337    338   332  0.09  0.62  0.56  0.56  0.31
26  241  26  0    333    329   325  0.09  0.62  0.55  0.59  0.27
27  253  27  0    330    327   325  0.09  0.61  0.55  0.66  0.30
28  248  28  0    336    333   325  0.10  0.60  0.53  0.66  0.28
29  246  29  0    349    347   325  0.09  0.59  0.50  0.64  0.32

 0 31943 320644.0 ! nice bought price perfect love quality 'm recommend -
   0-1 6518 32775.0 ! cover color love keyboard mac pro apple perfectly picture
     0-1-1 227 503.0 ! tons canon roomy lots ? cd cushion told samsonite
     0-1-3 6291 13447.0 ! mac keyboard protector logo protects screen cover retina air
   0-2 762 5585.0 camera cards lenses lens canon memory sd player flash dvd
     0-2-2 762 1564.0 lenses lens player sd organized grip trip small flash holds
   0-3 2893 18340.0 carry pockets comfortable pack back work plenty stuff compartments travel
     0-3-2 2893 7297.0 books pockets compartments plenty security shoulder room pack travel back
   0-4 2799 17684.0 & ; pocket power mouse cord room charger usb pockets
     0-4-3 2799 6614.0 ; & pocket mouse power sleeve drive cord room plenty
   0-5 4403 25550.0 strap pockets pocket shoulder zipper room inside carry inch handle
     0-5-1 114 233.0 broke zippers zipper coming poorly stitching poor months seams nylon
     0-5-2 

Exception ignored in: <_io.FileIO name='model/bags/ncrp/bags-mncrp-alp502010-eta5-gam0.01-epoch30/log' mode='wb' closefd=True>
